In [1]:
!pip install -q transformers
!pip install -q evaluate
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.3 MB/s eta 0:00:00


In [6]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/happy522/Argument-Mining/refs/heads/main/AM_gelabeled-2.csv")
df
keep_classes = [
    "Prämisse",
    "Behauptung",
    "Schlussfolgerung",
    "Nicht-argumentativer Text"
]

# Remove leading "/" in case your labels look like "/Zurückweisung"
df["Label"] = df["Label"].str.replace("/", "", regex=False).str.strip()

# Filter DataFrame
filtered_df = df[df["Label"].isin(keep_classes)]

# Classes you want to keep


# Reset index if needed
df = filtered_df.reset_index(drop=True)

#print(df)
#df.to_csv("new_arg_mining.csv")

CLASSES = [
    "Prämisse",
    "Behauptung",
    "Schlussfolgerung",
    "Nicht-argumentativer Text"
]

In [10]:
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from tqdm import tqdm

def knn_classification(embed_model_name):
    # Lade ein sentence-transformers Embedding-Modell (deutsches/mehrsprachiges)
    embed_model = SentenceTransformer(embed_model_name)

    # Train (1 Beispiel pro Klasse)
    examples = df.groupby("Label", group_keys=False).sample(n=1, random_state=42)
    X_train = embed_model.encode(
        examples["Komponente"].tolist(),
        convert_to_numpy=True,
        show_progress_bar=True  # tqdm progress bar
    )
    y_train = examples["Label"].tolist()

    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, y_train)

    # Inference
    X_test = embed_model.encode(
        df["Komponente"].tolist(),
        convert_to_numpy=True,
        show_progress_bar=True  # tqdm progress bar
    )
    preds = knn.predict(X_test)

    # Eval
    label2id = {l: i for i, l in enumerate(CLASSES)}
    y_true = df["Label"].map(label2id).astype(int).tolist()
    y_pred = [label2id.get(p, label2id["Nicht-argumentativer Text"]) for p in preds]

    #print(y_pred)
    print(classification_report(y_true, y_pred, target_names=CLASSES, zero_division=0))


In [7]:
knn_classification("sentence-transformers/distiluse-base-multilingual-cased-v2")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

[3, 0, 0, 0, 1, 2, 2, 2, 0, 2, 0, 3, 2, 2, 3, 3, 0, 0, 3, 0, 3, 1, 0, 0, 1, 1, 1, 3, 0, 3, 0, 3, 0, 0, 0, 3, 3, 3, 1, 2, 3, 2, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 3, 0, 3, 0, 3, 3, 3, 2, 3, 0, 3, 3, 0, 3, 3, 3, 0, 0, 0, 3, 3, 0, 0, 1, 0, 1, 2, 3, 3, 0, 2, 2, 3, 3, 2, 0, 0, 2, 3, 0, 2, 0, 0, 3, 3, 2, 1, 0, 0, 0, 1, 3, 3, 3, 2, 2, 0, 3, 1, 3, 3, 1, 1, 0, 0, 0, 0, 2, 2, 3, 0, 0, 0, 2, 0, 3, 3, 1, 0, 0, 1, 2, 3, 3, 0, 0, 2, 0, 2, 2, 0, 0, 3, 2, 3, 2, 3, 0, 3, 1, 0, 3, 3, 3, 0, 3, 0, 3, 2, 0, 3, 1, 3, 2, 2, 0, 3, 0, 3, 0, 1, 0, 3, 2, 0, 2, 0, 3, 2, 3, 3, 3, 3, 1, 1, 0, 2, 0, 0, 2, 0, 0, 2, 1, 0, 1, 0, 2, 0, 2, 2, 2, 1, 2, 0, 3, 0, 0, 3, 1, 2, 2, 1, 0, 3, 0, 2, 1, 2, 0, 1, 1, 2, 3, 3, 3, 2, 1, 3, 3, 0, 1, 2, 0, 3, 0, 3, 2, 3, 0, 2, 1, 2, 2, 2, 3, 0, 2, 0, 0, 3, 3, 0, 3, 2, 2, 3, 0, 0, 0, 0, 0, 2, 0, 3, 3, 2, 2, 3, 3, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 2, 1, 2, 3, 3, 2, 2, 2, 3, 3, 2, 0, 0, 0, 3, 0, 3, 3, 0, 2, 2, 0, 2, 2, 3, 0, 2, 0, 0, 0, 2, 3, 1, 1, 2, 3, 2, 0, 1, 3, 3, 2, 1, 2, 0, 1, 0, 3, 

In [11]:
knn_classification("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.38      0.23      0.28       193
               Behauptung       0.67      0.01      0.02       204
         Schlussfolgerung       0.05      0.22      0.08        18
Nicht-argumentativer Text       0.70      0.93      0.80       658

                 accuracy                           0.61      1073
                macro avg       0.45      0.35      0.29      1073
             weighted avg       0.63      0.61      0.55      1073



In [12]:
knn_classification("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.40      0.33      0.36       193
               Behauptung       0.40      0.09      0.15       204
         Schlussfolgerung       0.04      0.06      0.05        18
Nicht-argumentativer Text       0.72      0.93      0.81       658

                 accuracy                           0.65      1073
                macro avg       0.39      0.35      0.34      1073
             weighted avg       0.59      0.65      0.59      1073



In [13]:
knn_classification("distilbert-base-german-cased")

config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/270M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/479k [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.47      0.51      0.49       193
               Behauptung       0.18      0.08      0.11       204
         Schlussfolgerung       0.04      0.39      0.08        18
Nicht-argumentativer Text       0.85      0.78      0.81       658

                 accuracy                           0.59      1073
                macro avg       0.38      0.44      0.37      1073
             weighted avg       0.64      0.59      0.61      1073



In [14]:
knn_classification("T-Systems-onsite/cross-en-de-roberta-sentence-transformer")

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.27      0.53      0.36       193
               Behauptung       0.29      0.10      0.15       204
         Schlussfolgerung       0.02      0.06      0.03        18
Nicht-argumentativer Text       0.76      0.65      0.70       658

                 accuracy                           0.51      1073
                macro avg       0.33      0.33      0.31      1073
             weighted avg       0.57      0.51      0.52      1073



In [21]:
knn_classification("deepset/gbert-base")

config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.45      0.47      0.46       193
               Behauptung       0.40      0.05      0.09       204
         Schlussfolgerung       0.33      0.06      0.10        18
Nicht-argumentativer Text       0.76      0.97      0.85       658

                 accuracy                           0.69      1073
                macro avg       0.49      0.39      0.37      1073
             weighted avg       0.63      0.69      0.62      1073



In [15]:
knn_classification("sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.22      0.58      0.32       193
               Behauptung       0.33      0.20      0.25       204
         Schlussfolgerung       0.03      0.06      0.04        18
Nicht-argumentativer Text       0.79      0.50      0.61       658

                 accuracy                           0.45      1073
                macro avg       0.34      0.33      0.30      1073
             weighted avg       0.59      0.45      0.48      1073



In [16]:
knn_classification("intfloat/multilingual-e5-base")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.31      0.60      0.41       193
               Behauptung       0.40      0.17      0.24       204
         Schlussfolgerung       0.01      0.33      0.03        18
Nicht-argumentativer Text       0.91      0.28      0.43       658

                 accuracy                           0.32      1073
                macro avg       0.41      0.35      0.28      1073
             weighted avg       0.69      0.32      0.39      1073



In [17]:
knn_classification("intfloat/multilingual-e5-large")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.29      0.53      0.38       193
               Behauptung       0.40      0.23      0.29       204
         Schlussfolgerung       0.02      0.44      0.04        18
Nicht-argumentativer Text       0.95      0.28      0.43       658

                 accuracy                           0.32      1073
                macro avg       0.41      0.37      0.28      1073
             weighted avg       0.71      0.32      0.39      1073



In [20]:
knn_classification("distiluse-base-multilingual-cased-v1")

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

                           precision    recall  f1-score   support

                 Prämisse       0.38      0.22      0.28       193
               Behauptung       0.35      0.03      0.05       204
         Schlussfolgerung       0.04      0.44      0.07        18
Nicht-argumentativer Text       0.74      0.83      0.78       658

                 accuracy                           0.56      1073
                macro avg       0.38      0.38      0.30      1073
             weighted avg       0.59      0.56      0.54      1073

